# 导入模块

In [1]:
import numpy as np
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
import matplotlib.pyplot as plt
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")

import pymysql
from sqlalchemy import create_engine
import json
import datetime
from datetime import timedelta ,time, datetime
from dateutil.relativedelta import relativedelta
from jsonpath import jsonpath
import re
import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 连接数据库

In [3]:
def query(sql,
          host="rm-wz930e5269fur1ht1mo.mysql.rds.aliyuncs.com",
          user="wxz",
          password="5JRcY9SaiepVlIq7iuPo",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 获取数据

1.宽表整合：订单号、下单日期、收货地址是否为风险地址、年龄、gps定位的下单地址个数、在网时长、下单电话和收货电话是否一致、申请时是否上传身份证

In [4]:
sql = '''
select 
om.create_time, om.id as order_id, om.order_number, tol.delivery_province_name, tol.delivery_city_name, tol.delivery_county_name, tol.delivery_detail_address, tol.delivery_mobile, om.user_mobile, om.coordinate, tmu.id_card_num, top.total_describes, pa.name as activity_name
, tmu.true_name, tmu.id_card_back_url, tmu.id_card_front_url, om.merchant_name, tod.sku_attributes
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,om.status
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
-- ,oa.trace_id
from db_digua_business.t_order om 
left join db_digua_business.t_order_logistics tol on tol.order_id=om.id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_risk tor on tor.order_id = om.id
left join db_digua_business.t_order_details tod on tod.order_id = om.id
-- left join db_credit.order_association oa on oa.order_id=om.id
where date_format(om.create_time, '%Y-%m-%d')>='2024-01-01'
and date_format(om.create_time, '%Y-%m-%d')<='2025-03-31'
'''
df = query(sql)

In [5]:
sql_risk = ''' -- risk等级
select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected
from db_credit.risk
'''
df_risk = query(sql_risk)

In [6]:
sql3 = '''
SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)
df_re = df_risk_examine.copy()
df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date,errors="coerce")
df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

In [7]:
sql_ra = ''' -- 996强拒表
select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
'''
df_ra = query(sql_ra)
df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date,errors="coerce")
df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

In [8]:
sql_br = '''
select 
oa.order_number, rb.verify_rs
from db_credit.risk_bairong rb
left join db_credit.order_association oa on oa.trace_id=rb.trace_id
'''
df_br = query(sql_br)

# 数据去重

In [9]:
df2 = df.copy()
df2[df2.id_card_num=='362324198310085415']

,create_time,order_id,order_number,delivery_province_name,delivery_city_name,delivery_county_name,delivery_detail_address,delivery_mobile,user_mobile,coordinate,id_card_num,total_describes,activity_name,true_name,id_card_back_url,id_card_front_url,merchant_name,sku_attributes,status2,status,tips,trace_id
854800,2024-11-16 11:20:04,1732477,A20241116112004143,江西省,上饶市,铅山县,江西省-上饶市-铅山县-石塘镇112号,13576340044,13576340044,"117.842824,28.1146",362324198310085415,"[""17771888162订单A20241116112004143，msg=1"", ""177...",端内价格-全新苹果,甘奇,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,小蚂蚁租机,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",租赁中,4,None,5b56a75170c091c49a44fd23741c4e36
854843,2024-11-16 11:28:33,1732522,A20241116112832188,江西省,上饶市,铅山县,江西省-上饶市-铅山县-石塘镇112号,13576340044,13576340044,"117.842883,28.114526",362324198310085415,None,端内价格-全新苹果,甘奇,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,迪瓜优选,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",订单取消,13,[],5b56a75170c091c49a44fd23741c4e36
854852,2024-11-16 11:30:12,1732531,A20241116113011197,江西省,上饶市,铅山县,江西省-上饶市-铅山县-石塘镇112号,13576340044,13576340044,"117.842879,28.114545",362324198310085415,None,端内价格-全新苹果,甘奇,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,迪瓜优选,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",订单取消,13,[],5b56a75170c091c49a44fd23741c4e36


In [10]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文"])].index,inplace=True)

# 定义状态
df = df.merge(df_risk[['trace_id', 'status_r', 'union_rent_tag', 'union_rent_rejected']], on='trace_id', how='left').merge(df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'], right_on=['order_id', 'time_ra'], how='left')
df['是否机审强拒'] = np.where(df.status_r=='1', 1, 0)
df['是否出库前风控强拒'] = np.where((df.status_re=='1')|(df.status_ra=='1'), 1, 0)
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df.loc[:, '人审拒绝'] = np.where((df.进件=='进件')&(~df.电审拒绝原因.isna())&(df.status2=='已退款'), 1, 0)
df.loc[:, '出库'] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

# 获取需要的数据

In [11]:
df = clean.drop_merchant(df)
df_new = df[((df.是否机审强拒==1)|(df.是否出库前风控强拒==1)|(df.人审拒绝==1)|(df.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯']))|(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True))] # &(df.进件=='进件')
# df_new = df[(df.进件=='进件')]

In [12]:
# 1.宽表整合：订单号、下单日期、收货地址是否为风险地址、年龄、gps定位的下单地址个数、在网时长、下单电话和收货电话是否一致、申请时是否上传身份证
# 偏远地区(新疆，自治州，内蒙古，少数民族)特别，甘肃张掖市，黑龙江.公主岭市，四川.达州
df_new.loc[:, '是否风险地区'] = np.where((df_new.delivery_province_name.str.contains(r'新疆|内蒙古'))|((df_new.delivery_province_name.str.contains(r'甘肃|黑龙江|四川'))&(df_new.delivery_city_name.str.contains(r'张掖市|公主岭市|达州市'))), 1, 0)
df_new.loc[:, 'age'] = df_new.apply(lambda x: clean.get_age(x.id_card_num, x.下单日期), axis=1)
df_new.loc[:, '电话是否一致'] = np.where(df_new.user_mobile==df_new.delivery_mobile,1,0)
df_new.loc[:, '是否上传身份证'] = np.where((df_new.id_card_front_url.isna())&(df_new.id_card_back_url.isna()), 0 ,1)
df_new = df_new[['order_number', '下单日期', '是否风险地区', 'age', '电话是否一致', '是否上传身份证', 'coordinate', 'trace_id', 'merchant_name', '出库']]

In [13]:
df_coor = df_new.groupby('coordinate').agg({'order_number': 'count'}).rename(columns={'order_number': 'gps地址个数'})
df_merge = df_new.merge(df_coor, on='coordinate', how='left')

In [14]:
df_merge.columns

Index(['order_number', '下单日期', '是否风险地区', 'age', '电话是否一致', '是否上传身份证',
       'coordinate', 'trace_id', 'merchant_name', '出库', 'gps地址个数'],
      dtype='object')

In [15]:
# df_br.loc[:, '在网时长'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..name_rule'))
df_br.loc[:, '在网时长'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..TelPeriod.data.value'))
df_br.loc[:, '是否正常'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..rule_c_telstatus..name_rule'))
df_br.loc[:, '是否一致'] = df_br.verify_rs.apply(lambda x: None if x is None else jsonpath(json.loads(x), '$..rule_c_telcheck_s..name_rule'))
df_merges = df_merge.merge(df_br[['order_number', '在网时长', '是否正常', '是否一致']], on=['order_number'], how='left')


In [16]:
internet_dict = {
    '1': '[0, 6)',
    '2': '[6, 12)',
    '3': '[12, 24)',
    '4': '[24, +)]'
}
df_merges.loc[:, '在网时长_new'] = df_merges.在网时长.apply(lambda x: '无结果' if x is None or x==False or x is np.nan else internet_dict[str(x[0])])
df_merges.loc[:, '是否正常_new'] = np.where(df_merges.是否正常.str.contains('手机状态正常', regex=False), '正常', np.where(df_merges.是否正常.str.contains('手机状态异常:停机、销号等', regex=False), '异常', None))
df_merges.loc[:, '是否一致_new'] = np.where(df_merges.是否一致.str.contains('手机三要素简版验证一致', regex=False), '一致', np.where(df_merges.是否一致.str.contains('手机三要素简版验证不一致', regex=False), '不一致', None))
df_merges.loc[:, '是否推到小蚂蚁'] = np.where(df_merges.merchant_name=='小蚂蚁租机', 1, 0)
df_merges.loc[:, '小蚂蚁是否出库'] = np.where((df_merges.是否推到小蚂蚁==1)&(df_merges.出库==1), 1, 0)

In [17]:
df_merges = df_merges[['order_number', '下单日期', '是否风险地区', 'age', 'gps地址个数', '在网时长_new', '是否正常_new', '是否一致_new' , '电话是否一致', '是否上传身份证', '是否推到小蚂蚁', '小蚂蚁是否出库']]
with pd.ExcelWriter('F:/需求/总需求/手机三要素.xlsx', engine='xlsxwriter') as writer:
    df_merges.to_excel(writer, sheet_name='手机三要素', index=False)
df_merges

,order_number,下单日期,是否风险地区,age,gps地址个数,在网时长_new,是否正常_new,是否一致_new,电话是否一致,是否上传身份证,是否推到小蚂蚁,小蚂蚁是否出库
0,A2024010118455832,2024-01-01,0,37,1.0,"[24, +)]",正常,一致,1,0,0,0
1,A202401011102343,2024-01-01,0,22,1.0,"[24, +)]",正常,一致,0,0,0,0
2,A2024010111184212,2024-01-01,0,28,1.0,"[24, +)]",正常,一致,1,0,0,0
3,A2024010111320221,2024-01-01,0,30,1.0,"[24, +)]",正常,一致,1,1,0,0
4,A2024010111344424,2024-01-01,0,41,1.0,"[24, +)]",正常,一致,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
147527,A20250331152102311,2025-03-31,0,19,1.0,"[24, +)]",正常,一致,1,1,0,0
147528,A2025033117091292,2025-03-31,0,22,1.0,"[24, +)]",正常,不一致,0,1,0,0
147529,A20250331173122270,2025-03-31,0,28,1.0,无结果,正常,一致,0,1,1,1
147530,A20250331184959357,2025-03-31,0,19,1.0,"[0, 6)",正常,一致,1,1,0,0


In [18]:
df[df.order_number=='A2025031922361797']

,create_time,order_id,order_number,delivery_province_name,delivery_city_name,delivery_county_name,delivery_detail_address,delivery_mobile,user_mobile,coordinate,id_card_num,total_describes,activity_name,true_name,id_card_back_url,id_card_front_url,merchant_name,sku_attributes,status2,status,tips,trace_id,下单日期,状态编码,status_r,union_rent_tag,union_rent_rejected,status_re,time_ra,status_ra,是否机审强拒,是否出库前风控强拒,进件,电审拒绝原因,人审拒绝,出库
1017767,2025-03-19 22:36:18,2254756,A2025031922361797,广东省,深圳市,南山区,粤海街道科技路18号大冲新城花园 2 栋 1F11C,15520016157,15520016157,"113.950825,22.542356",52242720010818465X,"[""06762_正元订单A2025031922361797，msg=2"", ""发起修改套餐工...",端内价格-全新苹果,黄富华,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,https://diguazu.oss-cn-shenzhen.aliyuncs.com/u...,小蚂蚁租机,"[{""key"":""租赁方案"",""value"":""租完即送"",""type"":""text""},{...",租赁中,4,"[30天多头>9, 3日内GPS(100)米内下单人数:1, 3日内GPS(1000)米内下...",cda55521674fd12b34f4f45d5298a442,2025-03-19,7.0,1,None,None,NaN,NaT,NaN,1,0,进件,NaN,0,1
